# Chicago Train - TF Models

Apply tf ml to time series data

In [1]:
# Import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path

print(f'TensorFlow version {tf.__version__}')

TensorFlow version 2.12.0


In [2]:
# Load functions
def to_windows(dataset, length):
    dataset = dataset.window(length, shift=1, drop_remainder=True)
    return dataset.flat_map(lambda window_ds: window_ds.batch(length))

In [3]:
# Import data
path = Path("./ts_data/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])

df.columns = ["date", "day_type", "bus", "rail", "total"]
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)
df = df.drop_duplicates()

df

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917
...,...,...,...
2022-12-27,W,315154,202638
2022-12-28,W,321472,212073
2022-12-29,W,347158,231287


## Make a train-test data split

In [4]:
# Only use some sections of the data for the split and divide by 1M to get values closer to one
rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_valid = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":"2019-12"] / 1e6

rail_train.head()

date
2016-01-01    0.319835
2016-01-02    0.365509
2016-01-03    0.287661
2016-01-04    0.703185
2016-01-05    0.727716
Name: rail, dtype: float64

In [5]:
# Set and shuffle the windows
seq_length = 56
train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)

valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_valid.to_numpy(),
    targets=rail_valid[seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

## Make some models
### Start with a standard linear model


In [6]:
# Linear model with an early stopping callback
tf.random.set_seed(42)

# Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(1, input_shape=[seq_length])
])

# Callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=50,
    restore_best_weights=True
)

# Optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compile model
model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=['mae']
)

# Fit model
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=500,
    callbacks=[early_stopping_cb]
)

Epoch 1/500
33/33 [==============================] - 1s 6ms/step - loss: 0.1460 - mae: 0.4167 - val_loss: 0.0104 - val_mae: 0.1165
Epoch 2/500
33/33 [==============================] - 0s 4ms/step - loss: 0.0103 - mae: 0.1087 - val_loss: 0.0064 - val_mae: 0.0796
Epoch 3/500
33/33 [==============================] - 0s 6ms/step - loss: 0.0067 - mae: 0.0832 - val_loss: 0.0054 - val_mae: 0.0717
Epoch 4/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0063 - mae: 0.0800 - val_loss: 0.0081 - val_mae: 0.0958
Epoch 5/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0064 - mae: 0.0826 - val_loss: 0.0050 - val_mae: 0.0708
Epoch 6/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0053 - mae: 0.0720 - val_loss: 0.0051 - val_mae: 0.0712
Epoch 7/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0049 - mae: 0.0680 - val_loss: 0.0045 - val_mae: 0.0635
Epoch 8/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0049 - m

In [7]:
# Now do a simple RNN model with no differencing
model_rnn = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(56, input_shape=[None, 1]),
    tf.keras.layers.Dense(1)
])

# Callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=50,
    restore_best_weights=True
)

# Optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compile model
model_rnn.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=['mae']
)

# Fit model
history_rnn = model_rnn.fit(
    train_ds,
    validation_data=valid_ds,
    epochs=500,
    callbacks=[early_stopping_cb]
)

Epoch 1/500
33/33 [==============================] - 1s 13ms/step - loss: 0.0120 - mae: 0.1095 - val_loss: 0.0028 - val_mae: 0.0473
Epoch 2/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0042 - mae: 0.0588 - val_loss: 0.0023 - val_mae: 0.0426
Epoch 3/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0039 - mae: 0.0559 - val_loss: 0.0022 - val_mae: 0.0391
Epoch 4/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0034 - mae: 0.0514 - val_loss: 0.0025 - val_mae: 0.0400
Epoch 5/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0033 - mae: 0.0503 - val_loss: 0.0027 - val_mae: 0.0452
Epoch 6/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0032 - mae: 0.0498 - val_loss: 0.0023 - val_mae: 0.0368
Epoch 7/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0031 - mae: 0.0492 - val_loss: 0.0020 - val_mae: 0.0347
Epoch 8/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0031 - 

## Forecasting a Multivariate Time Series

In [8]:
# Forecast the rail time series using rail, bus, and type of day input
df_multivar = df[["bus", "rail"]] / 1e6 # Both bus and rail time series
df_multivar["next_day_type"] = df["day_type"].shift(-1) # We know tomorrow's day type
df_multivar = pd.get_dummies(df_multivar) # One-hot encode the day type

df_multivar.head() # Look at our dataframe

,bus,rail,next_day_type_A,next_day_type_U,next_day_type_W
date,,,,,
2001-01-01,0.297192,0.126455,False,False,True
2001-01-02,0.780827,0.501952,False,False,True
2001-01-03,0.824923,0.536432,False,False,True
2001-01-04,0.870021,0.550011,False,False,True
2001-01-05,0.890426,0.557917,True,False,False


In [9]:
# Now make a train-val-test split of the multivariate dataset
multivar_train = df_multivar["2016-01":"2018-12"]
multivar_valid = df_multivar["2019-01":"2019-05"]
multivar_test = df_multivar["2019-06":"2019-12"]

In [10]:
# And we window the datasets as before
seq_length = 56
train_multivar_ds = tf.keras.utils.timeseries_dataset_from_array(
    multivar_train.to_numpy(dtype=float),
    targets=multivar_train["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)

valid_multivar_ds = tf.keras.utils.timeseries_dataset_from_array(
    multivar_valid.to_numpy(dtype=float),
    targets=multivar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32
)

In [11]:
# Make a simple RNN model for the predictions
model_multivar = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(56, input_shape=[None, 5]),
    tf.keras.layers.Dense(1)
])

# Callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=50,
    restore_best_weights=True
)

# Optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compile model
model_multivar.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=['mae']
)

# Fit model
history_multivar = model_multivar.fit(
    train_multivar_ds,
    validation_data=valid_multivar_ds,
    epochs=500,
    callbacks=[early_stopping_cb]
)

Epoch 1/500
33/33 [==============================] - 1s 13ms/step - loss: 0.0400 - mae: 0.1963 - val_loss: 0.0014 - val_mae: 0.0391
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0025 - mae: 0.0535 - val_loss: 0.0014 - val_mae: 0.0407
Epoch 3/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0019 - mae: 0.0437 - val_loss: 8.7083e-04 - val_mae: 0.0306
Epoch 4/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0018 - mae: 0.0416 - val_loss: 9.6705e-04 - val_mae: 0.0346
Epoch 5/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0016 - mae: 0.0400 - val_loss: 0.0015 - val_mae: 0.0444
Epoch 6/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0016 - mae: 0.0392 - val_loss: 6.6908e-04 - val_mae: 0.0267
Epoch 7/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0014 - mae: 0.0373 - val_loss: 7.7457e-04 - val_mae: 0.0286
Epoch 8/500
33/33 [==============================] - 0s 7ms/step -

## Forecast Several Time Steps Ahead

Univariate or Multivariate, several time steps into the future.

### Method 1: Predict future values one time step at a time

In [12]:
X = rail_valid.to_numpy()[np.newaxis, :seq_length, np.newaxis]
print(X.shape)
X[0][0]

(1, 56, 1)


array([0.245852])

In [13]:
# Use univariate model RNN to make the predictions
for step_ahead in range(14):
    y_pred_one = model_rnn.predict(X)
    X = np.concatenate([X, y_pred_one.reshape(1, 1, 1)], axis=1)

print(X)

1/1 [==============================] - 0s 19ms/step
[[[0.245852  ]
  [0.573542  ]
  [0.627781  ]
  [0.628514  ]
  [0.348257  ]
  [0.260556  ]
  [0.66247   ]
  [0.701693  ]
  [0.685444  ]
  [0.690162  ]
  [0.679115  ]
  [0.325219  ]
  [0.250222  ]
  [0.705571  ]
  [0.720095  ]
  [0.728048  ]
  [0.734929  ]
  [0.700006  ]
  [0.312424  ]
  [0.236744  ]
  [0.401503  ]
  [0.712125  ]
  [0.718899  ]
  [0.7113    ]
  [0.590674  ]
  [0.296063  ]
  [0.221081  ]
  [0.67881   ]
  [0.579917  ]
  [0.097917  ]
  [0.243912  ]
  [0.648091  ]
  [0.34565   ]
  [0.250724  ]
  [0.718764  ]
  [0.734181  ]
  [0.727769  ]
  [0.732351  ]
  [0.655823  ]
  [0.338847  ]
  [0.251192  ]
  [0.704667  ]
  [0.698387  ]
  [0.728764  ]
  [0.746611  ]
  [0.687932  ]
  [0.362613  ]
  [0.256817  ]
  [0.519409  ]
  [0.725195  ]
  [0.717172  ]
  [0.727504  ]
  [0.702988  ]
  [0.328179  ]
  [0.240478  ]
  [0.680844  ]
  [0.72692209]
  [0.69115847]
  [0.71990317]
  [0.64429498]
  [0.35560134]
  [0.1912151 ]
  [0.5828681 ]
  [

### Method 2: Predict all values in one shot

In [15]:
# We need to remake the dataset distributions to predict 14 time steps in one shot
def split_inputs_and_targets(multivar_series, ahead=14, target_col=1):
    return multivar_series[:, :-ahead], multivar_series[:, -ahead:, target_col]

# And we window the datasets with the adjustments
seq_length = 56
ahead_train_ds = tf.keras.utils.timeseries_dataset_from_array(
    multivar_train.to_numpy(dtype=float),
    targets=None,
    sequence_length=seq_length + 14,
    batch_size=32,
    shuffle=True,
    seed=42
).map(split_inputs_and_targets)

ahead_valid_ds = tf.keras.utils.timeseries_dataset_from_array(
    multivar_valid.to_numpy(dtype=float),
    targets=None,
    sequence_length=seq_length + 14,
    batch_size=32
).map(split_inputs_and_targets)


In [16]:
# Make an RNN model that returns 14 values
model_ahead = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(56, input_shape=[None, 5]),
    tf.keras.layers.Dense(14) # One neuron for each ahead time step
])

# Callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=50,
    restore_best_weights=True
)

# Optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compile model
model_ahead.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=['mae']
)

# Fit model
history_ahead = model_ahead.fit(
    ahead_train_ds,
    validation_data=ahead_valid_ds,
    epochs=500,
    callbacks=[early_stopping_cb]
)

Epoch 1/500
33/33 [==============================] - 1s 12ms/step - loss: 0.0919 - mae: 0.3127 - val_loss: 0.0175 - val_mae: 0.1483
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0148 - mae: 0.1343 - val_loss: 0.0096 - val_mae: 0.1043
Epoch 3/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0093 - mae: 0.1045 - val_loss: 0.0063 - val_mae: 0.0847
Epoch 4/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0072 - mae: 0.0891 - val_loss: 0.0043 - val_mae: 0.0701
Epoch 5/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0060 - mae: 0.0791 - val_loss: 0.0035 - val_mae: 0.0628
Epoch 6/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0053 - mae: 0.0713 - val_loss: 0.0027 - val_mae: 0.0546
Epoch 7/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0049 - mae: 0.0667 - val_loss: 0.0024 - val_mae: 0.0510
Epoch 8/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0046 - 

In [18]:
# Make some predictions
X = multivar_valid.to_numpy(dtype=float)[np.newaxis, :seq_length]
Y_pred = model_ahead.predict(X)

1/1 [==============================] - 0s 121ms/step


In [19]:
print(Y_pred)
print(Y_pred.shape)

[[0.7095143  0.7302104  0.718292   0.68094325 0.3651237  0.260342
  0.6444582  0.71294636 0.71861243 0.7199055  0.67453754 0.3651618
  0.2604215  0.61715007]]
(1, 14)


## Forecasting Using Sequence-to-Sequence Models

I think this is the method used in the TensorFlow exam

### A quick primer on properly windowing the datasets

In [20]:
def to_windows(dataset, length):
    dataset = dataset.window(length, shift=1, drop_remainder=True)
    return dataset.flat_map(lambda window_ds: window_ds.batch(length))

In [21]:
# Make a dummy dataset and mess with it
my_series = tf.data.Dataset.range(7)
dataset = to_windows(to_windows(my_series, 3), 4)
list(dataset)

[<tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[0, 1, 2],
        [1, 2, 3],
        [2, 3, 4],
        [3, 4, 5]], dtype=int64)>,
 <tf.Tensor: shape=(4, 3), dtype=int64, numpy=
 array([[1, 2, 3],
        [2, 3, 4],
        [3, 4, 5],
        [4, 5, 6]], dtype=int64)>]

We have a dataset with two arrays, one being one time step ahead

In [22]:
# Now split the dataset into inputs and targets
dataset = dataset.map(lambda S: (S[:, 0], S[:, 1:]))
list(dataset)

[(<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 1, 2, 3], dtype=int64)>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[1, 2],
         [2, 3],
         [3, 4],
         [4, 5]], dtype=int64)>),
 (<tf.Tensor: shape=(4,), dtype=int64, numpy=array([1, 2, 3, 4], dtype=int64)>,
  <tf.Tensor: shape=(4, 2), dtype=int64, numpy=
  array([[2, 3],
         [3, 4],
         [4, 5],
         [5, 6]], dtype=int64)>)]

### Now use the Chicago Train Dataset

In [23]:
# Make a helper function
def to_seq2seq_dataset(series, seq_length=56, ahead=14, target_col=1, batch_size=32, shuffle=False, seed=None):
    ds = to_windows(tf.data.Dataset.from_tensor_slices(series), ahead+1)
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 1]))
    if shuffle:
        ds = ds.shuffle(8 * batch_size, seed=seed)
    return ds.batch(batch_size)

In [25]:
# Create the datasets
seq2seq_train = to_seq2seq_dataset(multivar_train.to_numpy(dtype=float), shuffle=True, seed=42)
seq2seq_valid = to_seq2seq_dataset(multivar_valid.to_numpy(dtype=float))

In [26]:
# Make a model to handle the sequences
model_seq2seq = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

# Callback
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=50,
    restore_best_weights=True
)

# Optimizer
opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)

# Compile model
model_seq2seq.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=opt,
    metrics=['mae']
)

# Fit model
history_seq2seq = model_seq2seq.fit(
    seq2seq_train,
    validation_data=seq2seq_valid,
    epochs=500,
    callbacks=[early_stopping_cb]
)


Epoch 1/500
33/33 [==============================] - 1s 15ms/step - loss: 0.1014 - mae: 0.3465 - val_loss: 0.0303 - val_mae: 0.1946
Epoch 2/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0198 - mae: 0.1566 - val_loss: 0.0166 - val_mae: 0.1385
Epoch 3/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0145 - mae: 0.1317 - val_loss: 0.0131 - val_mae: 0.1225
Epoch 4/500
33/33 [==============================] - 0s 9ms/step - loss: 0.0122 - mae: 0.1186 - val_loss: 0.0110 - val_mae: 0.1106
Epoch 5/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0107 - mae: 0.1093 - val_loss: 0.0098 - val_mae: 0.1024
Epoch 6/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0096 - mae: 0.1022 - val_loss: 0.0089 - val_mae: 0.0966
Epoch 7/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0089 - mae: 0.0965 - val_loss: 0.0080 - val_mae: 0.0905
Epoch 8/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0082 - 

In [28]:
# Make some predictions
X = multivar_valid.to_numpy(dtype=float)[np.newaxis, :seq_length]
y_pred_14 = model_seq2seq.predict(X)

1/1 [==============================] - 0s 169ms/step


In [29]:
print(X.shape)
X

(1, 56, 5)


array([[[0.248879, 0.245852, 0.      , 0.      , 1.      ],
        [0.591006, 0.573542, 0.      , 0.      , 1.      ],
        [0.664442, 0.627781, 0.      , 0.      , 1.      ],
        [0.668812, 0.628514, 1.      , 0.      , 0.      ],
        [0.444434, 0.348257, 0.      , 1.      , 0.      ],
        [0.318623, 0.260556, 0.      , 0.      , 1.      ],
        [0.723023, 0.66247 , 0.      , 0.      , 1.      ],
        [0.784085, 0.701693, 0.      , 0.      , 1.      ],
        [0.747949, 0.685444, 0.      , 0.      , 1.      ],
        [0.74785 , 0.690162, 0.      , 0.      , 1.      ],
        [0.763718, 0.679115, 1.      , 0.      , 0.      ],
        [0.382464, 0.325219, 0.      , 1.      , 0.      ],
        [0.301855, 0.250222, 0.      , 0.      , 1.      ],
        [0.75659 , 0.705571, 0.      , 0.      , 1.      ],
        [0.783113, 0.720095, 0.      , 0.      , 1.      ],
        [0.800537, 0.728048, 0.      , 0.      , 1.      ],
        [0.800348, 0.734929, 0.      , 0

In [30]:
print(y_pred_14.shape)
y_pred_14[0][-1]

(1, 56, 14)


array([0.7327291 , 0.7334709 , 0.7400911 , 0.7337377 , 0.41355518,
       0.30880693, 0.66812885, 0.74340844, 0.74282396, 0.75133526,
       0.7113839 , 0.40962434, 0.3026625 , 0.692943  ], dtype=float32)

## Different Model Types with Sequence-to-Sequence Multivariate Data

In [31]:
# Prepare the data
def to_seq2seq_dataset(series, seq_length=56, ahead=14, target_col=1, batch_size=32, shuffle=False, seed=None):
    ds = to_windows(tf.data.Dataset.from_tensor_slices(series), ahead+1)
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 1]))
    if shuffle:
        ds = ds.shuffle(8 * batch_size, seed=seed)
    return ds.batch(batch_size)

In [33]:
seq2seq_train = to_seq2seq_dataset(multivar_train.to_numpy(dtype=float), shuffle=True, seed=42)
seq2seq_valid = to_seq2seq_dataset(multivar_valid.to_numpy(dtype=float))

In [34]:
for example in seq2seq_train.take(1):
    print(example[0])

tf.Tensor(
[[[0.873275 0.804156 0.       0.       1.      ]
  [0.849532 0.775198 0.       0.       1.      ]
  [0.852594 0.787476 1.       0.       0.      ]
  ...
  [0.415172 0.376896 0.       1.       0.      ]
  [0.393858 0.410925 0.       0.       1.      ]
  [0.769179 0.735491 0.       0.       1.      ]]

 [[0.898889 0.774877 0.       0.       1.      ]
  [0.85105  0.763128 0.       0.       1.      ]
  [0.86767  0.786281 0.       0.       1.      ]
  ...
  [0.500775 0.532343 0.       1.       0.      ]
  [0.409978 0.430982 0.       1.       0.      ]
  [0.394914 0.389143 0.       0.       1.      ]]

 [[0.883059 0.75475  1.       0.       0.      ]
  [0.503983 0.421193 0.       1.       0.      ]
  [0.39361  0.327173 0.       0.       1.      ]
  ...
  [0.862743 0.782134 0.       0.       1.      ]
  [0.856722 0.782591 0.       0.       1.      ]
  [0.85172  0.821158 0.       0.       1.      ]]

 ...

 [[0.78606  0.777948 0.       0.       1.      ]
  [0.789537 0.831925 0.     

### RNN

In [40]:
epochs = 250

loss_huber = tf.keras.losses.Huber()
opt_adam = tf.keras.optimizers.legacy.Adam()

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_mae",
    patience=20,
    restore_best_weights=True
)


In [41]:
# Set up an RNN Model
model_mulvar_rnn = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None, 5]),
    tf.keras.layers.SimpleRNN(56, return_sequences=True),
    tf.keras.layers.Dense(14)
])

model_mulvar_rnn.compile(
    loss= loss_huber,
    optimizer=opt_adam,
    metrics=['mae']
)

model_mulvar_rnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_5 (SimpleRNN)    (None, None, 56)          3472      
                                                                 
 dense_7 (Dense)             (None, None, 14)          798       
                                                                 
Total params: 4,270
Trainable params: 4,270
Non-trainable params: 0
_________________________________________________________________


In [42]:
history_mulvar_rnn = model_mulvar_rnn.fit(
    seq2seq_train,
    validation_data=seq2seq_valid,
    epochs= epochs,
    callbacks=[early_stopping_cb]
)

Epoch 1/250
33/33 [==============================] - 1s 15ms/step - loss: 0.0772 - mae: 0.2970 - val_loss: 0.0205 - val_mae: 0.1544
Epoch 2/250
33/33 [==============================] - 0s 9ms/step - loss: 0.0152 - mae: 0.1259 - val_loss: 0.0099 - val_mae: 0.0967
Epoch 3/250
33/33 [==============================] - 0s 9ms/step - loss: 0.0094 - mae: 0.0903 - val_loss: 0.0071 - val_mae: 0.0744
Epoch 4/250
33/33 [==============================] - 0s 8ms/step - loss: 0.0074 - mae: 0.0759 - val_loss: 0.0062 - val_mae: 0.0677
Epoch 5/250
33/33 [==============================] - 0s 8ms/step - loss: 0.0065 - mae: 0.0695 - val_loss: 0.0056 - val_mae: 0.0636
Epoch 6/250
33/33 [==============================] - 0s 9ms/step - loss: 0.0058 - mae: 0.0659 - val_loss: 0.0054 - val_mae: 0.0638
Epoch 7/250
33/33 [==============================] - 0s 9ms/step - loss: 0.0054 - mae: 0.0633 - val_loss: 0.0049 - val_mae: 0.0598
Epoch 8/250
33/33 [==============================] - 0s 9ms/step - loss: 0.0050 - 

### GRU

In [43]:
model_mulvar_gru = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None, 5]),
    tf.keras.layers.GRU(56, return_sequences=True),
    tf.keras.layers.Dense(14)
])

model_mulvar_gru.compile(
    loss= loss_huber,
    optimizer=opt_adam,
    metrics=['mae']
)

model_mulvar_gru.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_1 (GRU)                 (None, None, 56)          10584     
                                                                 
 dense_8 (Dense)             (None, None, 14)          798       
                                                                 
Total params: 11,382
Trainable params: 11,382
Non-trainable params: 0
_________________________________________________________________


In [44]:
history_mulvar_gru = model_mulvar_gru.fit(
    seq2seq_train,
    validation_data=seq2seq_valid,
    epochs= epochs,
    callbacks=[early_stopping_cb]
)

Epoch 1/250
33/33 [==============================] - 3s 28ms/step - loss: 0.0433 - mae: 0.2150 - val_loss: 0.0120 - val_mae: 0.1203
Epoch 2/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0094 - mae: 0.1061 - val_loss: 0.0086 - val_mae: 0.0988
Epoch 3/250
33/33 [==============================] - 1s 16ms/step - loss: 0.0072 - mae: 0.0903 - val_loss: 0.0069 - val_mae: 0.0850
Epoch 4/250
33/33 [==============================] - 1s 16ms/step - loss: 0.0060 - mae: 0.0797 - val_loss: 0.0058 - val_mae: 0.0755
Epoch 5/250
33/33 [==============================] - 1s 16ms/step - loss: 0.0053 - mae: 0.0720 - val_loss: 0.0050 - val_mae: 0.0676
Epoch 6/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0048 - mae: 0.0668 - val_loss: 0.0044 - val_mae: 0.0612
Epoch 7/250
33/33 [==============================] - 1s 16ms/step - loss: 0.0041 - mae: 0.0593 - val_loss: 0.0040 - val_mae: 0.0561
Epoch 8/250
33/33 [==============================] - 1s 16ms/step - loss: 0.

### LSTM

In [45]:
model_mulvar_lstm = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None, 5]),
    tf.keras.layers.LSTM(56, return_sequences=True),
    tf.keras.layers.Dense(14)
])

model_mulvar_lstm.compile(
    loss= loss_huber,
    optimizer=opt_adam,
    metrics=['mae']
)

model_mulvar_lstm.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 56)          13888     
                                                                 
 dense_9 (Dense)             (None, None, 14)          798       
                                                                 
Total params: 14,686
Trainable params: 14,686
Non-trainable params: 0
_________________________________________________________________


In [46]:
history_mulvar_lstm = model_mulvar_lstm.fit(
    seq2seq_train,
    validation_data=seq2seq_valid,
    epochs= epochs,
    callbacks=[early_stopping_cb]
)

Epoch 1/250
33/33 [==============================] - 3s 31ms/step - loss: 0.0377 - mae: 0.2116 - val_loss: 0.0175 - val_mae: 0.1433
Epoch 2/250
33/33 [==============================] - 1s 18ms/step - loss: 0.0141 - mae: 0.1384 - val_loss: 0.0124 - val_mae: 0.1182
Epoch 3/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0099 - mae: 0.1070 - val_loss: 0.0089 - val_mae: 0.1002
Epoch 4/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0081 - mae: 0.0948 - val_loss: 0.0075 - val_mae: 0.0899
Epoch 5/250
33/33 [==============================] - 1s 18ms/step - loss: 0.0071 - mae: 0.0862 - val_loss: 0.0071 - val_mae: 0.0860
Epoch 6/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0064 - mae: 0.0800 - val_loss: 0.0060 - val_mae: 0.0762
Epoch 7/250
33/33 [==============================] - 1s 19ms/step - loss: 0.0058 - mae: 0.0741 - val_loss: 0.0053 - val_mae: 0.0695
Epoch 8/250
33/33 [==============================] - 1s 18ms/step - loss: 0.

### RNN with CNN

In [47]:
model_mulvar_rnn_cnn = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None, 5]),
    tf.keras.layers.SimpleRNN(56, return_sequences=True),
    tf.keras.layers.Conv1D(32, 5, 1, padding="same"),
    tf.keras.layers.Dense(14)
])

model_mulvar_rnn_cnn.compile(
    loss= loss_huber,
    optimizer=opt_adam,
    metrics=['mae']
)

model_mulvar_rnn_cnn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_6 (SimpleRNN)    (None, None, 56)          3472      
                                                                 
 conv1d (Conv1D)             (None, None, 32)          8992      
                                                                 
 dense_10 (Dense)            (None, None, 14)          462       
                                                                 
Total params: 12,926
Trainable params: 12,926
Non-trainable params: 0
_________________________________________________________________


In [48]:
history_mulvar_rnn_cnn = model_mulvar_rnn_cnn.fit(
    seq2seq_train,
    validation_data=seq2seq_valid,
    epochs= epochs,
    callbacks=[early_stopping_cb]
)

Epoch 1/250
33/33 [==============================] - 1s 17ms/step - loss: 0.0384 - mae: 0.1771 - val_loss: 0.0062 - val_mae: 0.0769
Epoch 2/250
33/33 [==============================] - 0s 11ms/step - loss: 0.0046 - mae: 0.0643 - val_loss: 0.0043 - val_mae: 0.0590
Epoch 3/250
33/33 [==============================] - 0s 11ms/step - loss: 0.0036 - mae: 0.0549 - val_loss: 0.0038 - val_mae: 0.0541
Epoch 4/250
33/33 [==============================] - 0s 10ms/step - loss: 0.0033 - mae: 0.0516 - val_loss: 0.0037 - val_mae: 0.0533
Epoch 5/250
33/33 [==============================] - 0s 11ms/step - loss: 0.0031 - mae: 0.0502 - val_loss: 0.0035 - val_mae: 0.0502
Epoch 6/250
33/33 [==============================] - 0s 10ms/step - loss: 0.0030 - mae: 0.0493 - val_loss: 0.0035 - val_mae: 0.0506
Epoch 7/250
33/33 [==============================] - 0s 12ms/step - loss: 0.0029 - mae: 0.0474 - val_loss: 0.0034 - val_mae: 0.0508
Epoch 8/250
33/33 [==============================] - 0s 10ms/step - loss: 0.